In [2]:
from Bio.PDB import *
import pandas as pd
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
import csv
import numpy as np
from Bio.PDB.Polypeptide import three_to_one, one_to_three                           
import math

load affinity bench excel file

In [3]:
def load_pd(fName):
    df = pd.read_csv(fName)
    return df

abIn = "C:/Users/amber/Study_Documents/intern/inno/ppi with binding affinity/affinitybench/affinity_bench_v2.csv"
ppi = load_pd(abIn)
ppi = np.array(ppi[2:])
print("Size of the data:", ppi.shape)
# print(ppi)


Size of the data: (135, 6)


Originaly, there are 135 protein pairs

In [4]:
pid_set = []

for i in range(135):
# for i in range(10):
#     print(ppi[i,:])
    ent_name = ppi[i,0][:4]
#     .lower()
#   set of pid
    if ent_name not in pid_set:
        pid_set.append(ent_name)

        # remove more than protein complex with more than two chains
        chains = ppi[i,0].split('_')[1:]
        #print(chains)
        chain_list = chains[0].split(":")
        # print(chain_list)
        if len(chain_list[0]) > 1 or len(chain_list[1]) >1:
            pid_set.remove(ent_name) 
print(len(pid_set))

94


In [5]:
def download_all_pdb(pid_set):
   pdbl = PDBList()
   for elem in pid_set:
        print(elem)
        assert(len(elem) ==4)
        pdbl.retrieve_pdb_file(elem, pdir = './affinitybench/PDBs/', file_format='pdb')

download_all_pdb(pid_set)  

1KXQ
Structure exists: './affinitybench/PDBs/pdb1kxq.ent' 
2I25
Structure exists: './affinitybench/PDBs/pdb2i25.ent' 
1ACB
Structure exists: './affinitybench/PDBs/pdb1acb.ent' 
1AVX
Structure exists: './affinitybench/PDBs/pdb1avx.ent' 
1BRS
Structure exists: './affinitybench/PDBs/pdb1brs.ent' 
1BUH
Structure exists: './affinitybench/PDBs/pdb1buh.ent' 
1BVN
Structure exists: './affinitybench/PDBs/pdb1bvn.ent' 
1DFJ
Structure exists: './affinitybench/PDBs/pdb1dfj.ent' 
1EAW
Structure exists: './affinitybench/PDBs/pdb1eaw.ent' 
1EMV
Structure exists: './affinitybench/PDBs/pdb1emv.ent' 
1F34
Structure exists: './affinitybench/PDBs/pdb1f34.ent' 
1FLE
Structure exists: './affinitybench/PDBs/pdb1fle.ent' 
1GXD
Structure exists: './affinitybench/PDBs/pdb1gxd.ent' 
1JIW
Structure exists: './affinitybench/PDBs/pdb1jiw.ent' 
1JTG
Structure exists: './affinitybench/PDBs/pdb1jtg.ent' 
1MAH
Structure exists: './affinitybench/PDBs/pdb1mah.ent' 
1OPH
Structure exists: './affinitybench/PDBs/pdb1oph.ent

In [6]:
def get_seq(ent_name, chain_ID0, chain_ID1, seq_dict):
#     print(ent_names, chain_ID0, chain_ID1)
    parser = PDBParser()
    
    
    structure = parser.get_structure(ent_name, './affinitybench/PDBs/pdb'+ ent_name.lower() + '.ent')
    #     assert(len(structure) == 1)
    model = structure[0]

    w_name0 = ent_name+'_'+chain_ID0
    if w_name0 not in seq_dict:
        w_seq0 = ''

        chain0 = model[chain_ID0]
        ppb = PPBuilder()
        for pp in ppb.build_peptides(chain0):
            w_seq0 += pp.get_sequence()

        seq_dict[w_name0] = w_seq0
        
        
    w_name1 = ent_name+'_'+chain_ID1
    if w_name1 not in seq_dict:
        w_seq1 = ''

        chain1 = model[chain_ID1]
        ppb = PPBuilder()
        for pp in ppb.build_peptides(chain1):
            w_seq1 += pp.get_sequence()

        seq_dict[w_name1] = w_seq1
    return seq_dict

In [7]:
def get_dg(ent_name, chain_ID0, chain_ID1, dg, dg_info):
    
    w_name0 = ent_name+'_'+chain_ID0
    w_name1 = ent_name+'_'+chain_ID1
   
    info = w_name0 + " " +  w_name1 + " " + str(dg)
    dg_info.append(info)
    return dg_info

In [9]:
seq_dict = {}
dg_dict ={}
dg_info = []

f1 = open("./affinitybench/affinitybench.seq.txt", "w")
f2 = open("./affinitybench/affinitybench.dg.txt", "w")

for i in range(135):
# for i in range(10):
#     print(ppi[i,:])
    ent_name = ppi[i,0][:4]
#     .lower()
    if ent_name in pid_set:
        chains = ppi[i,0].split('_')[1:]
        chain_list = chains[0].split(":")
        chain_ID0 = chain_list[0]
        chain_ID1 = chain_list[1]

        dg = ppi[i,4]
        #print(i, ppi[i,], chain_ID0, chain_ID1, dg)
        
        get_seq(ent_name, chain_ID0, chain_ID1, seq_dict)
        get_dg(ent_name, chain_ID0, chain_ID1, dg, dg_info)
    
for elem in seq_dict:
    print(elem, seq_dict[elem], file=open("./affinitybench/affinitybench.seq.txt", "a"))

for elem in dg_info:
    print(elem, file=open("./affinitybench/affinitybench.dg.txt", "a"))

f1.close()
f2.close()

Only 95 protein pairs are recorded. Others are with multiple chains. 

One of example is 1BVK_DE:F, with chain D and E are Fv Hulys11 and Chain F is HEW lysozyme. see https://www.rcsb.org/3d-view/1BVK.
